# “Learning about Machine Learning with CRIM”


**Abstract**

In this tutorial-essay I will consider how we can use machine learning, speciﬁcally dimensionality reduction and embedding methods, with the CRIM corpus. The guiding question is how style can be modeled quantitatively. Building both on music-theoretical conceptualization and machine learning techniques, it will be demonstrated that unsupervised clustering can serve to some degree as a proxy for stylistic similarity. The CRIM data set provides an ideal case study that will also point to some shortcomings of the computational methodology that can only be resolved by a critical view, drawing on musicological expertise and close-reading of sources.


## Introduction: setting the scope

## Setup and obtaining the data

We begin by installing the CRIM intervals library. 

```{bash}
pip install --upgrade --force-reinstall git+https://github.com/HCDigitalScholarship/intervals.git@main 
```


Next we import all libraries and modules that we will need for our subsequent analyses.

In [1]:
abs_path = "/home/fmoss/GitHub/fabianmoss/crim-haverford/" # absolute path to my working directory

import os, glob # file I/O
from tqdm import tqdm # status bar for loops
import re # regular expressions
import requests # to download files

import intervals as ci # crim intervals
# import music21 as m21
import pandas as pd # to work with tabular data

import matplotlib.pyplot as plt # plots, plots, plots

# dimensionality reduction
from sklearn.decomposition import PCA
from umap import UMAP

In [4]:
# us = m21.environment.UserSettings()
# us.getSettingsPath()
# us["musescoreDirectPNGPath"] = "/home/fmoss/.local/bin/mscore"

# import notebook
# notebook.nbextensions.check_nbextension('usability/sphinx-markdown', user=True)

# E = notebook.nbextensions.EnableNBExtensionApp()
# E.toggle_nbextension('usability/sphinx-markdown/main')

# c = m21.chord.Chord(["C4", "E4", "G4"])

# hexa = ci.analysis.neoRiemannian.completeHexatonic(c, simplifyEnharmonics=True)
# hexa

# for chord in hexa:
#     chord.duration=m21.duration.Duration(1.)

# s = m21.stream.Stream(hexa)

# s.show("text")

# s.show() # --> this doesn't work yet

We now access the CRIM corpus and download it to our working directory, so that we have to download it only once.
First we get a list of the URLs pointing to each piece in the corpus following the instructions [here](https://github.com/RichardFreedman/CRIM_JHUB/blob/main/Make-me-a-Corpus.ipynb) and [here](https://github.com/RichardFreedman/CRIM_JHUB/blob/main/CRIM_04b_Cadences_Corpus.ipynb).

In [2]:
raw_prefix = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"
URL = "https://api.github.com/repos/CRIM-Project/CRIM-online/git/trees/990f5eb3ff1e9623711514d6609da4076257816c"
piece_json = requests.get(URL).json()
piece_list = [raw_prefix + p["path"] for p in piece_json["tree"]]

The variable `piece_list` now contains all URLs and names of files in the CRIM corpus. We can inspect the first 5 items: 

In [3]:
piece_list[:5]

['https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_1.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_2.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_3.mei',
 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_4.mei']

In total, we have `len(piece_list)` pieces: 

In [4]:
len(piece_list)

307

There are 307 files in total. Downloading the files takes a certain amount of time. To speed this up, we save all files in `piece_list` in our local directory.

First, we create a new directory `data/` but only if it does not already exist.

In [5]:
d = "data/"
if not os.path.exists(d):
    os.makedirs(d)

Next, we iterate over `piece_list`, request the file from the server and save it to that directory.

In [6]:
for piece in tqdm(piece_list[:10]):
    filename = piece.split("/")[-1] # only the part after the last '/' is the filename    
    with open(d + filename, "wb") as f:
        # if not os.path.exists(d + filename):
        r = requests.get(piece)
        f.write(r.content)

100%|███████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]


We create a new list `local_files` containing all local file paths and names.

In [2]:
local_files = glob.glob("data/*.mei")

So now we have a list of file names pointing to MEI files in our local `data/` directory. At a closer look you'll see that some of them end in something like `0001_1.mei` but a few others end in `0001.mei`. There is a pattern to this. The files without the trailing digit are 'wrappers' that bind all movements (indexed 1 through 9) of a particular mass (indexed 0001 through 9999) together. Since these wrappers do not contain any notes or cadences (those are stored in the MEI files of the respective movements), we'll filter them out. 

Fortunately, this is very easy since the filenames are chosen systematically. We only need to remove all files from the `local_files` list that have a file name ending in `_d.mei`, where `d` stands for any integer from 1 to 9.

In [3]:
local_files = [ f  for f in local_files if re.match(r".+_\d.mei$", f) ]

What happened here? We defined a pattern according to which we were able to remove the wrapper file names. This pattern is here expressed as a **regular expression**: `r".+_\d.mei$"`

Let's take it apart to understand how it works.
As you probably now, strings in Python are surrounded by either one or two quotation marks (`'` or `"`). The `r` prefixed to the expression tells the interpreter that the string enclosed in quotes is a regular expression and that the characters have to be interpreted accordingly. 

Next, we see a period `.`. This symbol stands for "any character" in a regular expression. The following `+` means "one or more", so that the combination `.+` stands for a sequence of any characters of length at least 1. With this, we capture the part of the filename preceding the underscore `_`. 

Since the pattern differs towards the end of the file names, we can also view it from the end: The `$` sign marks the end of the string, so that everything to its left has to come just before. Since we are dealing with MEI files, each file name ends with `.mei`, which is exactly what we see before the `$`. 

Now the crucial part. The 'wrapper' files do not have an underscore followed by a single-digit integer. We can use this information and represent that integer with `\d`. 

Consequently, filenames **not** following this pattern (not being captured by the regular expression) will not be taken into account. In English, we could read the list comprehension for `local_files` as: "make a list of filenames where each filename conforms to the pattern defined by the regular expression".

These are the 5 first movements in the list:

In [4]:
local_files[:5]

['data/CRIM_Mass_0001_5.mei',
 'data/CRIM_Mass_0002_2.mei',
 'data/CRIM_Mass_0001_1.mei',
 'data/CRIM_Mass_0001_2.mei',
 'data/CRIM_Mass_0002_1.mei']

As it turns out, they are not in lexicographical order. This can be easily fixed with the following line of code.

In [5]:
local_files = sorted(local_files)

In [7]:
local_files[:5]

['data/CRIM_Mass_0001_1.mei',
 'data/CRIM_Mass_0001_2.mei',
 'data/CRIM_Mass_0001_3.mei',
 'data/CRIM_Mass_0001_4.mei',
 'data/CRIM_Mass_0001_5.mei']

In [8]:
len(local_files)

8

Apparently, there are "only" 220 individual mass movements. 

## Transforming the data

Now that we have all files nicely stored in our local directory, it is finally time to access them and transform them from the MEI encoding into a tabular format. The CRIM intervals library (imported as `ci`, see above) provides a convenient way to do so: we create a `corpus` by passing a list of files to the `CorpusBase` object.

In [9]:
corpus = ci.CorpusBase([abs_path + f for f in local_files] )

Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0001_1.mei
Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0001_2.mei
Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0001_3.mei
Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0001_4.mei
Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0001_5.mei
Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0002_1.mei
Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0002_2.mei
Successfully imported /home/fmoss/GitHub/fabianmoss/crim-haverford/data/CRIM_Mass_0002_3.mei


The `corpus` object now provides a range of convenient methods to access and transform the data further.
For instance, we can access all cadences from the first score `.scores[0]` in the corpus using the `.cadences()` method as follows:

In [10]:
corpus.scores[0].cadences()

,CadType,LeadingTones,CVFs,Low,RelLow,Tone,RelTone,TSig,Measure,Beat,Sounding,Progress,SinceLast,ToNext
28.0,Clausula Vera,0,CT,D3,-P4,D,P5,4/2,4,3.0,4.0,0.118644,28.0,12.0
40.0,Authentic,0,TCB,D3,-P4,D,P5,4/2,6,1.0,4.0,0.169492,12.0,16.0
56.0,Authentic,0,CB,B-3,m3,D,P5,4/2,8,1.0,4.0,0.237288,16.0,16.0
72.0,Authentic,1,CTB,G3,P1,G,P8,8/2,10,1.0,4.0,0.305085,16.0,56.0
128.0,Authentic,0,CB,G3,P1,G,P8,4/2,16,1.0,3.0,0.542373,56.0,8.0
136.0,Clausula Vera,0,TC,G3,P1,G,P8,4/2,17,1.0,4.0,0.576271,8.0,8.0
144.0,Authentic,0,CTB,D3,-P4,D,P5,4/2,18,1.0,4.0,0.610169,8.0,48.0
192.0,Clausula Vera,1,CT,G3,P1,G,P8,4/2,23,1.0,3.0,0.813559,48.0,16.0
208.0,Clausula Vera,0,TC,G3,P1,G,P8,4/2,25,1.0,4.0,0.881356,16.0,28.0
236.0,Authentic,1,CTB,G3,P1,G,P8,10/2,28,3.0,4.0,1.000000,28.0,16.0


## The vector-space model

**In the _vector-space model_, counts of terms are represented in a high-dimensional abstract space. Imagine that terms are stored in a vocabulary with $V$ entries, then the (relative) frequency of these terms in a document (here: in a mass movement) is represented by a real number.**

### $n$-gram models

The concept of $n$-grams is useful to describe sequences of different lengths. 

It comes from the field of _Formal Language Theory_ (FLT). Imagine we have two sequences (e.g., melodies), 
$$s_1 = n_1 n_2 \ldots n_k$$
and 
$$s_2 = n_1 n_2 \ldots n_l,$$
and both melodies are taken from a corpus within the same style. We can then as questions like, which melody is more likely (= more frequent)? 

$$P(s_1) \geq P(s_2).$$

Moreover, this model $n$-gram models let us ask questions like "Given a particular sequence (melody) $s=n_1\ldots n_t$, what is the most likely next note?:"

$$P(n_{t+1}| n_1 \ldots n_t)$$

In the above expression, the probability of the note $n_{t+1}$ depends on the entire "history" of the previous notes in the melody. For a handful of notes, this is not a problem, but it can become difficult to calculate when sequences get very long. This is where the so-called _Markov assumption_ enters the stage:

$$P(n_{t+1}| n_1 \ldots n_t) \approx P(n_{t+1}| n_t).$$

It states that the probability of the next note in the melody, $n_{t+1}$, given the entire previous melody $n_1 \ldots n_t$ can be **approximated** by just looking at the probability of the next note $n_{t+1}$ given the last note $n_t$. We approximate entire melodies by only looking at pairs of notes! These are called **bigrams**. While this seems to be a drastic reduction of complexity (and it is!), it does work quite well in practice.

The $n$ in $n$-gram model determines the length of the history taken into account:
- **unigrams** ($n=1$) take a history of length 0 into account (these are just the raw frequencies of occurrence)
- **bigrams** ($n=2$) take a history of one note into account
- **trigrams** ($n=3$) take a history of two notes into account
- etc.

In [15]:
corpus.batch(func=ci.ImportedPiece.ngrams)[0]

,4_3,4_2,4_1,3_2,3_1,2_1,Composer,Title
0.0,"3_1, 3_-4, 5","5_1, 5_-4, 8","8_1, 8_-4, 10","3_1, 3_-2, 4","6_1, 6_-2, 6","4_1, 4_1, 3",Pierre Colin,Missa Confitemini: Kyrie
6.0,"3_-4, 5_4, 3","5_-4, 8_4, 5","8_-4, 10_4, 8","3_-2, 4_2, 3","6_-2, 6_2, 6","4_1, 3_1, 4",Pierre Colin,Missa Confitemini: Kyrie
8.0,"5_4, 3_-2, 5","8_4, 5_-2, 8","10_4, 8_-2, 10","4_2, 3_2, 4","6_2, 6_2, 6","3_1, 4_3, 3",Pierre Colin,Missa Confitemini: Kyrie
12.0,"3_-2, 5_Held, 4","5_-2, 8_Held, 7","8_-2, 10_Held, 9","3_2, 4_-2, 4","6_2, 6_-2, 6","4_3, 3_-2, 3",Pierre Colin,Missa Confitemini: Kyrie
16.0,"5_Held, 4_Held, 3","8_Held, 7_Held, 6","10_Held, 9_Held, 10","4_-2, 4_Held, 3","6_-2, 6_-2, 8","3_-2, 3_-2, 4",Pierre Colin,Missa Confitemini: Kyrie
...,...,...,...,...,...,...,...,...
229.0,"7_3, 5_Held, 4",NaN,NaN,"4_1, 2_-2, 3","7_1, 6_-2, 7",NaN,Pierre Colin,Missa Confitemini: Kyrie
230.0,"5_Held, 4_Held, 3","6_-2, 8_4, 5","10_-2, 11_Held, 10","2_-2, 3_-2, 4","6_-2, 7_-2, 8","5_2, 4_Held, 3",Pierre Colin,Missa Confitemini: Kyrie
231.0,"4_Held, 3_-2, 5",NaN,NaN,"3_-2, 4_2, 4","7_-2, 8_2, 7",NaN,Pierre Colin,Missa Confitemini: Kyrie
231.5,"3_-2, 5_Held, 5",NaN,NaN,"4_2, 4_1, 4","8_2, 7_1, 6",NaN,Pierre Colin,Missa Confitemini: Kyrie


### Term frequencies

### Document frequencies

### Term frequency - inverse document frequency (TF-IDF)

In [19]:
def count_all(df, normalize=False):
    s = pd.concat([df[col] for col in df.columns])
    return s.value_counts(normalize=normalize)

In [20]:
count_all(corpus.scores[2].getNoteRest(), normalize=False)

AttributeError: 'ImportedPiece' object has no attribute 'getNoteRest'

In [ ]:
counts = pd.DataFrame([ count_all( corpus.scores[i].getNoteRest()) for i in range(len(corpus.scores)) ]).reset_index(drop=True)
counts = counts.fillna(1)# fill NaN values
del counts["Rest"]
counts = counts.iloc[:,:20]
counts

In [ ]:
import numpy as np

dia = ["B-"] + list("FCGDAEB") + ["Rest"]

cmap = plt.get_cmap('tab20')
colors = cmap(np.linspace(0, 1, len(dia)))

c = [colors[dia.index(l)] for l in counts.idxmax(axis=1).apply(lambda x: x[:-1] if x[-1] != "t" else x).values]

In [ ]:
counts = counts.div(counts.sum(axis=1), axis=0)

In [ ]:
X = counts.values

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, metric="cosine", perplexity=25)

In [ ]:
X__ = tsne.fit_transform(X)

In [ ]:
plt.scatter(X__[:,0], X__[:,1], alpha=.75, zorder=3, c=c)
plt.show()

In [ ]:
corpus.scores[3].analyses["note_list"][4].note

## Dimensionality reduction

TEXT

We start of with our data in $\text

### The idea

### Principal Components Analysis (PCA): a simple and popular method

In [21]:
pca_reducer = PCA()

In [ ]:
X_ = pca_reducer.fit_transform(X)

In [ ]:
plt.scatter(X_[:,0], X_[:,1], alpha=.75, zorder=3, c=c)
plt.axhline(0, lw=.5, c="k")
plt.axvline(0, lw=.5, c="k")
plt.show()

### Uniform Manifold Approximation & Projection (UMAP): a complex and popular method

In [17]:
reducer = UMAP()

## On style

Meyer quote